In [1]:
# !pip install catboost

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 65.9 MB 9.6 MB/s eta 0:00:011
     |████████████████████████████████| 12.9 MB 15.3 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11430 sha256=29aa84b171fd01706a8d19844dc2cde136bab1ae72d36ee3afcc22b0b948804c
  Stored in directory: /home/tione/.cache/pip/wheels/f4/d2/64/0dac67a06d2fbcfdc9a26fe0cfaa7b3ac86c6b593f8f0d2880
Successfully built retrying


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime, time, gc
# from utils import distance, haversine, standard, pad_seq 
from scipy.stats import skew, kurtosis
from catboost import CatBoostClassifier
from zipfile import ZipFile
from collections import Counter 
from sklearn.metrics import roc_auc_score as auc
# import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
# !pip install tables

In [3]:
train_fea = pd.read_hdf("data/train_1kw_fea.hdf", 'df')
test_fea = pd.read_hdf("data/test_1kw_fea.hdf", 'df') 

uid_emb = pd.read_hdf("data/feature_uid_embed.hdf", 'df')
ua_pagerank = pd.read_hdf("data/feature_uq_pagerank.hdf", 'df')

train_fea = train_fea.merge(uid_emb, on='uid', how='left')
test_fea = test_fea.merge(uid_emb, on='uid', how='left')

train_fea = pd.concat([train_fea, ua_pagerank.iloc[:-1000000]], axis=1)
test_fea = pd.concat([test_fea, ua_pagerank.iloc[-1000000:]], axis=1)

In [4]:
X_train = train_fea[train_fea["pt_d"]<=6].copy()
y_train = X_train["label"].astype('int32')
X_valid = train_fea[train_fea["pt_d"]>6]
y_valid = X_valid["label"].astype('int32')

In [6]:
cate_cols = [
       'uid', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
       'app_first_class', 'app_second_class', 'city',
       'device_name', 'career', 'gender', 'net_type',
       'residence', 'his_on_shelf_time', "communication_onlinerate", 
       'up_membership_grade', 'consume_purchase', 'indu_name',
]

use_cols = list( set(train_fea.columns) - set(['label', 'cnt', 'random_sector', 'pt_d',
        'uid_task_id_ctr', 'uid_adv_id_ctr', 'uid_spread_app_id_ctr', 'uid_adv_prim_id_ctr', 'uid_dev_id_ctr',
        'uid_app_second_class_ctr']))

print(len(use_cols), len(cate_cols))  

167 23


In [8]:
#线下验证
cate_fea = []
clf = CatBoostClassifier(iterations=5000, learning_rate=0.05, 
                         loss_function='Logloss', cat_features=cate_fea, 
                         verbose=True, eval_metric='AUC', counter_calc_method='Full',
                         depth=15, max_leaves=63, 
                         grow_policy='Lossguide',
                         min_data_in_leaf=20, 
                         reg_lambda=1.0,
#                          one_hot_max_size=4,
#                          task_type='GPU',
                         metric_period=50)
clf.fit(
    X_train[use_cols], y_train.astype('int32'), 
    eval_set=[(X_valid[use_cols], y_valid.astype('int32'))],
    early_stopping_rounds=50,
    verbose=True,
    use_best_model=True,
)

y_predprob = clf.predict_proba(X_valid[use_cols])[:, 1] 

y_pre = clf.predict_proba(test_fea[use_cols])[:, 1]  
auc_score =roc_auc_score(y_valid, y_predprob)
print("AUC Score (Valid): %f" % auc_score) 

0:	test: 0.7223297	best: 0.7223297 (0)	total: 3s	remaining: 4h 9m 58s
50:	test: 0.7955648	best: 0.7955648 (50)	total: 2m 43s	remaining: 4h 23m 54s
100:	test: 0.7993696	best: 0.7993974 (98)	total: 6m 13s	remaining: 5h 2m 9s
150:	test: 0.8001231	best: 0.8001231 (150)	total: 9m 46s	remaining: 5h 14m 5s
200:	test: 0.8009079	best: 0.8009152 (199)	total: 13m 16s	remaining: 5h 16m 51s
250:	test: 0.8015050	best: 0.8015050 (250)	total: 16m 40s	remaining: 5h 15m 29s
300:	test: 0.8017928	best: 0.8017928 (300)	total: 19m 59s	remaining: 5h 12m 7s
350:	test: 0.8021389	best: 0.8021389 (350)	total: 23m 15s	remaining: 5h 8m 5s
400:	test: 0.8023396	best: 0.8023396 (400)	total: 26m 13s	remaining: 5h 46s
450:	test: 0.8025096	best: 0.8025096 (449)	total: 29m 10s	remaining: 4h 54m 15s
500:	test: 0.8025947	best: 0.8026029 (496)	total: 31m 48s	remaining: 4h 45m 39s
550:	test: 0.8027442	best: 0.8027442 (550)	total: 34m 29s	remaining: 4h 38m 32s
600:	test: 0.8027626	best: 0.8027725 (595)	total: 36m 58s	remainin

In [9]:
result = pd.DataFrame({
        'id': np.load("data/test_id.npy"), 
        'probability': y_pre, 
    })

result.to_csv("submission.csv", index=False)  

In [10]:
result.head()

,id,probability
0,1,0.008360
1,2,0.097068
2,3,0.141274
3,4,0.011108
4,5,0.046296
